# Title: Deriving Insights from Netflix Dataset:  ETL + Visualization Project (Python, SQL & Tableau).

#### Author: Salim Sharif. 

## Step 1: Data Extraction and Familiarization. 

In [2]:
#importing libraries
#!pip install kaggle
import kaggle

In [3]:
#extracting data from kaggle API
!kaggle datasets download shivamb/netflix-shows -f netflix_titles.csv

Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0
netflix_titles.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#extracting the zip file from the local drive
import zipfile

zip_ref = zipfile.ZipFile('netflix_titles.csv.zip')
zip_ref.extractall() #extract file to dir
zip_ref.close() #close the file

In [2]:
#Read the CSV file and familizing with the dataset
import pandas as pd

df = pd.read_csv('netflix_titles.csv')
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


## Step 2: Data Transformation and Loading.

In [8]:
#Setting up a database (MSSQL) to store the cleaned and transformed data.

In [5]:
#Connecting into Server Engine
import sqlalchemy as sal
engine = sal.create_engine('mssql://SALIM\SQLEXPRESS/Personal_Projects?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [7]:
#loading data into MSSQL
df.to_sql('netflix_raw', con= conn , index = False , if_exists = 'replace')
conn.close()

In [8]:
#investigating the junky char in the titles
df[df.show_id== 's2639'] #junky char due to foreign language

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2638,s2639,Movie,الف مبروك,Ahmed Nader Galal,"Ahmed Helmy, Laila Ezz El Arab, Mahmoud El Fis...",Egypt,"April 25, 2020",2009,TV-14,115 min,"Comedies, Dramas, International Movies","On his wedding day, an arrogant, greedy accoun..."


In [22]:
len(df)

8807

In [ ]:
#Creating a optimized table in SQ Lwith correct dtype

In [13]:
columns = df.columns
columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [35]:
#determining max char length for each column

# Iterate over the DataFrame's columns
for column in df.columns:
    try:
    # Calculate the maximum string length for the current column
        max_length = max(df[column].dropna().str.len())
        print(f"{column}: {max_length}")

    except (ValueError, TypeError,AttributeError):
        # If an error occurs (e.g., due to incompatible data types), skip the column
        print(f"Error processing column: {column}")
    

show_id: 5
type: 7
title: 104
director: 208
cast: 771
country: 123
date_added: 19
Error processing column: release_year
rating: 8
duration: 10
listed_in: 79
description: 248


In [36]:
#Loading data into new created sql table

In [38]:
import sqlalchemy as sal
engine = sal.create_engine('mssql://SALIM\SQLEXPRESS/Personal_Projects?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

#loading data into MSSQL
df.to_sql('netflix_raw', con= conn , index = False , if_exists = 'append')
conn.close()

In [39]:
df[df.show_id == 's2037'] #It has special char in title

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2036,s2037,Movie,#Alive,Cho Il,"Yoo Ah-in, Park Shin-hye",South Korea,"September 8, 2020",2020,TV-MA,99 min,"Horror Movies, International Movies, Thrillers","As a grisly virus rampages a city, a lone man ..."


In [8]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64